In [79]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
import json
f = open('api/out (2)')
items_list = json.load(f)

found = []
error_404 = []
error_429 = []
matched_names = []
for k,v in items_list.items():
    if type(v) is  list:
        found.append(k)
        matched_names.append(k)
    if v == 429:
        error_429.append(k)
    if v == 404:
        error_404.append(k)
print('Matched:', len(found))
print('No search result:', len(error_404))
print('Exceeded rate limit:', len(error_429))
print(len(matched_names))

print(error_404)
#print(items_list.keys())


Matched: 88
No search result: 56
Exceeded rate limit: 48
88
['McCormick Red 1 Pint Food Color', 'Folgers Classic Roast Regular Urn Coffee - 6.3 Oz.', 'Dickinsons Seedless Blackberry Preserves - 1 oz Jars', 'Welchs Grape Red Sparkling Juice - 24.5 Oz.', 'cup portion 1.5oz translucent (728544)', 'Beef Ground Chuck Angus B119 Fresh', 'Kraft Jello Americana Tapioca Pudding - 24 Oz. - 12 Pouches Per Case', 'Popcorn Cheddar Skinny Pack - 3.9 oz.', 'Eastern Bromley Decaffeinated Bags Tea - 1 Oz.', 'cup clear 16-18 oz flexible clear (1)', 'Monin Hazelnut Organic Flavored Syrup - 750 ml.', 'Flavor Fresh Mayonnaise Packet - 12 Gr.', 'Paper Pro Plates, 6in. Dia, Round, White', 'MUSTARD IND YELLOW 5.5 GRAM PACKET', 'Splenda No Calorie Sweetener Packet - 1 Grm.', 'Portion Pac Honey - 9 Grm.', 'Portion Pac Tartar Sauce - 9 Grm.', 'Gold Medal Restaurant Ground Black Pepper - 5 Pound', 'Black Pepper Pure Ground - 1 Lb.', 'Tomato Ketchup Dispenser Pouch - 1.5 Gal.', 'Paper Hot Dog White Medium Weight T

In [76]:
# Match back to matched.csv
import pandas as pd
random_df = pd.read_csv('api/random.csv', dtype=str)
random_df = random_df[random_df['source_name'].isin(matched_names)].reset_index()
random_df.drop_duplicates(keep=False, inplace=True)
random_df.drop(random_df.columns[:2], axis=1, inplace=True)

random_df.to_csv('api/matched_47.csv', index=False)

In [78]:
# Explore available columns in data
columns = ['source_name']

for k,v in items_list.items():
    if type(v) is list:
        for _ in v:
            for _k, _v in _.items():
                if _k not in columns:
                    columns.append(_k)

# Create item list
results = []
for k,v in items_list.items():
    if type(v) is list:
        for _ in v:
            default_dic = {}
            for c in columns:
                default_dic[c] = ''
            default_dic['source_name'] = k
            for _k, _v in _.items():
                default_dic[_k] = _v
            results.append(default_dic)
enriched = pd.DataFrame(results)
enriched['dest_UPC'] = 'UPC-' + enriched['upc']
enriched.to_csv('api/enriched.csv', index=False)

In [82]:
# Get data for these names from all-merged
import productmatching
base_folder = 'aws/dev-dashy-refined-datasets/'
df_all_merged = productmatching.read_file_from_csv(base_folder + "analytics/products", "all-merged.csv")
df_all_merged = df_all_merged[df_all_merged['name'].isin(matched_names)].reset_index()
#df_all_merged.to_csv('match_source.csv')

/var/folders/rj/rrq458bj1mb9fpw62rmd9yb80000gn/T/ipykernel_52740/1823137753.py:4: DtypeWarning: Columns (17,18,19,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df_all_merged = productmatching.read_file_from_csv(base_folder + "analytics/products", "all-merged.csv")


read 68136  rows from aws/dev-dashy-refined-datasets/analytics/products all-merged.csv


In [ ]:
# Normalise enriched files to same format


In [62]:
tmp = enriched.merge(random_df,how='inner',left_on=['source_name'],right_on=['source_name'])
#tmp = tmp[['source_name', 'title', 'description', 'source_brand', 'source_pack_count', 'source_unit_size',\
#    'source_unit_measure','confidence','dest_UPC_x', 'dest_UPC_y' ]]
tmp.to_csv('api/tmp.csv')


In [61]:
tmp = enriched.merge(random_df,how='inner',left_on=['dest_UPC'],right_on=['dest_UPC'])
tmp.to_csv('api/upc_matc.csv', index=False)


In [66]:
columns

['source_name',
 'ean',
 'title',
 'description',
 'upc',
 'brand',
 'model',
 'color',
 'size',
 'dimension',
 'weight',
 'category',
 'currency',
 'lowest_recorded_price',
 'highest_recorded_price',
 'images',
 'offers',
 'elid',
 'asin',
 'gtin']